.

Data_Engineering_TIL(20201211)

study_program : 컨테이너 오케스트레이션 쿠버네티스 살펴보기

실습 시 참고한 URL : https://github.com/subicura/workshop-k8s-basic

- 실습환경 셋팅

ec2를 생성하고 ssh 접속해서 아래와 같이 명령어를 실행한다.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ sudo yum update -y

[ec2-user@ip-10-0-1-231 ~]$ sudo yum install jq -y

[ec2-user@ip-10-0-1-231 ~]$ sudo amazon-linux-extras install docker

[ec2-user@ip-10-0-1-231 ~]$ sudo service docker start

[ec2-user@ip-10-0-1-231 ~]$ sudo usermod -a -G docker ec2-user

[ec2-user@ip-10-0-1-231 ~]$ exit

# ssh 재접속

[ec2-user@ip-10-0-1-231 ~]$ docker info
Client:
 Debug Mode: false

Server:
 Containers: 0
  Running: 0
  Paused: 0
  Stopped: 0
 Images: 0
 Server Version: 19.03.13-ce
 Storage Driver: overlay2
  Backing Filesystem: xfs
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 runc version: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 init version: de40ad0 (expected: fec3683)
 Security Options:
  seccomp
   Profile: default
 Kernel Version: 4.14.193-149.317.amzn2.x86_64
 Operating System: Amazon Linux 2
 OSType: linux
 Architecture: x86_64
 CPUs: 2
 Total Memory: 1.909GiB
 Name: ip-10-0-1-231.ap-northeast-2.compute.internal
 ID: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 Docker Root Dir: /var/lib/docker
 Debug Mode: false
 Registry: https://index.docker.io/v1/
 Labels:
 Experimental: false
 Insecure Registries:
  127.0.0.0/8
 Live Restore Enabled: false

[ec2-user@ip-10-0-1-231 ~]$ curl -sfL https://get.k3s.io | sh -
[INFO]  Finding release for channel stable
[INFO]  Using v1.19.4+k3s1 as release
[INFO]  Downloading hash https://github.com/rancher/k3s/releases/download/v1.19.4+k3s1/sha256sum-amd64.txt
[INFO]  Downloading binary https://github.com/rancher/k3s/releases/download/v1.19.4+k3s1/k3s
[INFO]  Verifying binary download
[INFO]  Installing k3s to /usr/local/bin/k3s
[INFO]  Creating /usr/local/bin/kubectl symlink to k3s
[INFO]  Creating /usr/local/bin/crictl symlink to k3s
[INFO]  Skipping /usr/local/bin/ctr symlink to k3s, command exists in PATH at /usr/bin/ctr
[INFO]  Creating killall script /usr/local/bin/k3s-killall.sh
[INFO]  Creating uninstall script /usr/local/bin/k3s-uninstall.sh
[INFO]  env: Creating environment file /etc/systemd/system/k3s.service.env
[INFO]  systemd: Creating service file /etc/systemd/system/k3s.service
[INFO]  systemd: Enabling k3s unit
Created symlink from /etc/systemd/system/multi-user.target.wants/k3s.service to /etc/systemd/system/k3s.service.
[INFO]  systemd: Starting k3s
    
[ec2-user@ip-10-0-1-231 ~]$ sudo chown ec2-user:ec2-user /etc/rancher/k3s/k3s.yaml

- 기본명령어 실습

get을 이용해서 각종정보를 조회해본다.

get은 object들을 list로 조회한다.

In [ ]:
# 쿠버네티스 클러스터 현황확인
# ec2 한대밖에없어서 하나만 보인다.
# 실제 프로덕션 환경에서는 여러대를 클러스터링해서 사용한다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get nodes
NAME                                            STATUS   ROLES    AGE   VERSION
ip-10-0-1-231.ap-northeast-2.compute.internal   Ready    master   36s   v1.19.4+k3s1

# -o wide 옵션을 주어 상세내용을 볼 수도 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get nodes -o wide
NAME                                            STATUS   ROLES    AGE   VERSION        INTERNAL-IP   EXTERNAL-IP   OS-IMAGE         KERNEL-VERSION                  CONTAINER-RUNTIME
ip-10-0-1-231.ap-northeast-2.compute.internal   Ready    master   23m   v1.19.4+k3s1   10.0.1.231    <none>        Amazon Linux 2   4.14.193-149.317.amzn2.x86_64   containerd://1.4.1-k3s1

# -o yaml 옵션을 주면 완전 자세한 내용을 확인할 수 있다.
# 쿠버네티스는 어떤형태의 상태를 관리하는 것이 목적인데 이런 상태를 체크하기에 적합한 옵션이다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get nodes -o yaml
    
# pod 현황확인
# 아직 만든게 없어서 노 리소스로 나온다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get pod
No resources found in default namespace.

[ec2-user@ip-10-0-1-231 ~]$ kubectl get replicaset
No resources found in default namespace.

[ec2-user@ip-10-0-1-231 ~]$ kubectl get deployment
No resources found in default namespace.

# 쿠버네티스에서 사용하는 서비스가 하나 보인다.
# 이거는 kubectl이 api 통신을 하기위해서 필요해서 떠있는 것이다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get service
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   9m46s

[ec2-user@ip-10-0-1-231 ~]$ kubectl get ingress
Warning: extensions/v1beta1 Ingress is deprecated in v1.14+, unavailable in v1.22+; use networking.k8s.io/v1 Ingress
No resources found in default namespace.

# 전체 오브젝트는 kubectl api-resources 명령어로 조회가 가능하다.
# 아래와 같은 리소스들이 복잡하게 얽혀서 쿠버네티스 클러스터가 동작하는 것이다.
# 하지만 실제로 사용자가 사용하는건 위에서 확인한 몇가지 것들 위주다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl api-resources
NAME                              SHORTNAMES   APIGROUP                       NAMESPACED   KIND
bindings                                                                      true         Binding
componentstatuses                 cs                                          false        ComponentStatus
configmaps                        cm                                          true         ConfigMap
endpoints                         ep                                          true         Endpoints
events                            ev                                          true         Event
limitranges                       limits                                      true         LimitRange
namespaces                        ns                                          false        Namespace
nodes                             no                                          false        Node
persistentvolumeclaims            pvc                                         true         PersistentVolumeClaim
persistentvolumes                 pv                                          false        PersistentVolume
pods                              po                                          true         Pod
podtemplates                                                                  true         PodTemplate
replicationcontrollers            rc                                          true         ReplicationController
resourcequotas                    quota                                       true         ResourceQuota
secrets                                                                       true         Secret
serviceaccounts                   sa                                          true         ServiceAccount
services                          svc                                         true         Service
mutatingwebhookconfigurations                  admissionregistration.k8s.io   false        MutatingWebhookConfiguration
validatingwebhookconfigurations                admissionregistration.k8s.io   false        ValidatingWebhookConfiguration
customresourcedefinitions         crd,crds     apiextensions.k8s.io           false        CustomResourceDefinition
apiservices                                    apiregistration.k8s.io         false        APIService
controllerrevisions                            apps                           true         ControllerRevision
daemonsets                        ds           apps                           true         DaemonSet
deployments                       deploy       apps                           true         Deployment
replicasets                       rs           apps                           true         ReplicaSet
statefulsets                      sts          apps                           true         StatefulSet
tokenreviews                                   authentication.k8s.io          false        TokenReview
localsubjectaccessreviews                      authorization.k8s.io           true         LocalSubjectAccessReview
selfsubjectaccessreviews                       authorization.k8s.io           false        SelfSubjectAccessReview
selfsubjectrulesreviews                        authorization.k8s.io           false        SelfSubjectRulesReview
subjectaccessreviews                           authorization.k8s.io           false        SubjectAccessReview
horizontalpodautoscalers          hpa          autoscaling                    true         HorizontalPodAutoscaler
cronjobs                          cj           batch                          true         CronJob
jobs                                           batch                          true         Job
certificatesigningrequests        csr          certificates.k8s.io            false        CertificateSigningRequest
leases                                         coordination.k8s.io            true         Lease
endpointslices                                 discovery.k8s.io               true         EndpointSlice
events                            ev           events.k8s.io                  true         Event
ingresses                         ing          extensions                     true         Ingress
helmchartconfigs                               helm.cattle.io                 true         HelmChartConfig
helmcharts                                     helm.cattle.io                 true         HelmChart
addons                                         k3s.cattle.io                  true         Addon
nodes                                          metrics.k8s.io                 false        NodeMetrics
pods                                           metrics.k8s.io                 true         PodMetrics
ingressclasses                                 networking.k8s.io              false        IngressClass
ingresses                         ing          networking.k8s.io              true         Ingress
networkpolicies                   netpol       networking.k8s.io              true         NetworkPolicy
runtimeclasses                                 node.k8s.io                    false        RuntimeClass
poddisruptionbudgets              pdb          policy                         true         PodDisruptionBudget
podsecuritypolicies               psp          policy                         false        PodSecurityPolicy
clusterrolebindings                            rbac.authorization.k8s.io      false        ClusterRoleBinding
clusterroles                                   rbac.authorization.k8s.io      false        ClusterRole
rolebindings                                   rbac.authorization.k8s.io      true         RoleBinding
roles                                          rbac.authorization.k8s.io      true         Role
priorityclasses                   pc           scheduling.k8s.io              false        PriorityClass
csidrivers                                     storage.k8s.io                 false        CSIDriver
csinodes                                       storage.k8s.io                 false        CSINode
storageclasses                    sc           storage.k8s.io                 false        StorageClass
volumeattachments                              storage.k8s.io                 false        VolumeAttachment

# pod, replicaset, deployment, service 한꺼번에 조회
[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   20m

# json으로 나온 결과를 파이프로 받아서 jq라는 얘가 다시 그거를 처리하는 명령어이다.
# 현재 이노드의 네임과 cpu 메모리 정보 등을 확인할 수 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get nodes -o json | jq ".items[] | {name:.metadata.name} + .status.capacity"
{
  "name": "ip-10-0-1-231.ap-northeast-2.compute.internal",
  "cpu": "2",
  "ephemeral-storage": "31444972Ki",
  "hugepages-1Gi": "0",
  "hugepages-2Mi": "0",
  "memory": "2002012Ki",
  "pods": "110"
}

- describe 명령어 실습

상세정보를 조회할때 사용한다.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ kubectl get nodes
NAME                                            STATUS   ROLES    AGE   VERSION
ip-10-0-1-231.ap-northeast-2.compute.internal   Ready    master   39m   v1.19.4+k3s1

[ec2-user@ip-10-0-1-231 ~]$ kubectl describe node ip-10-0-1-231.ap-northeast-2.compute.internal
Name:               ip-10-0-1-231.ap-northeast-2.compute.internal
Roles:              master
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=k3s
                    beta.kubernetes.io/os=linux
                    k3s.io/hostname=ip-10-0-1-231.ap-northeast-2.compute.internal
                    k3s.io/internal-ip=10.0.1.231
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=ip-10-0-1-231.ap-northeast-2.compute.internal
                    kubernetes.io/os=linux
                    node-role.kubernetes.io/master=true
                    node.kubernetes.io/instance-type=k3s
Annotations:        flannel.alpha.coreos.com/backend-data: {"VtepMAC":"xxxxxxxxxxxxxxxxxxxx"}
                    flannel.alpha.coreos.com/backend-type: vxlan
                    flannel.alpha.coreos.com/kube-subnet-manager: true
                    flannel.alpha.coreos.com/public-ip: 10.0.1.231
                    k3s.io/node-args: ["server"]
                    k3s.io/node-config-hash: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx====
                    k3s.io/node-env: {"K3S_DATA_DIR":"/var/lib/rancher/k3s/data/xxxxxxxxxxxxxxxxxxxxxxxxxx"}
                    node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Fri, 11 Dec 2020 08:09:46 +0000
Taints:             <none>
Unschedulable:      false
Lease:
  HolderIdentity:  ip-10-0-1-231.ap-northeast-2.compute.internal
  AcquireTime:     <unset>
  RenewTime:       Fri, 11 Dec 2020 08:49:26 +0000
Conditions:
  Type                 Status  LastHeartbeatTime                 LastTransitionTime                Reason                       Message
  ----                 ------  -----------------                 ------------------                ------                       -------
  NetworkUnavailable   False   Fri, 11 Dec 2020 08:10:02 +0000   Fri, 11 Dec 2020 08:10:02 +0000   FlannelIsUp                  Flannel is running on this node
  MemoryPressure       False   Fri, 11 Dec 2020 08:45:54 +0000   Fri, 11 Dec 2020 08:09:45 +0000   KubeletHasSufficientMemory   kubelet has sufficient memory available
  DiskPressure         False   Fri, 11 Dec 2020 08:45:54 +0000   Fri, 11 Dec 2020 08:09:45 +0000   KubeletHasNoDiskPressure     kubelet has no disk pressure
  PIDPressure          False   Fri, 11 Dec 2020 08:45:54 +0000   Fri, 11 Dec 2020 08:09:45 +0000   KubeletHasSufficientPID      kubelet has sufficient PID available
  Ready                True    Fri, 11 Dec 2020 08:45:54 +0000   Fri, 11 Dec 2020 08:09:56 +0000   KubeletReady                 kubelet is posting ready status
Addresses:
  InternalIP:  10.0.1.231
  Hostname:    ip-10-0-1-231.ap-northeast-2.compute.internal
Capacity:
  cpu:                2
  ephemeral-storage:  31444972Ki
  hugepages-1Gi:      0
  hugepages-2Mi:      0
  memory:             2002012Ki
  pods:               110
Allocatable:
  cpu:                2
  ephemeral-storage:  30589668738
  hugepages-1Gi:      0
  hugepages-2Mi:      0
  memory:             2002012Ki
  pods:               110
System Info:
  Machine ID:                 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  System UUID:                xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  Boot ID:                    xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  Kernel Version:             4.14.193-149.317.amzn2.x86_64
  OS Image:                   Amazon Linux 2
  Operating System:           linux
  Architecture:               amd64
  Container Runtime Version:  containerd://1.4.1-k3s1
  Kubelet Version:            v1.19.4+k3s1
  Kube-Proxy Version:         v1.19.4+k3s1
PodCIDR:                      10.42.0.0/24
PodCIDRs:                     10.42.0.0/24
ProviderID:                   k3s://ip-10-0-1-231.ap-northeast-2.compute.internal
Non-terminated Pods:          (5 in total)
  Namespace                   Name                                      CPU Requests  CPU Limits  Memory Requests  Memory Limits  AGE
  ---------                   ----                                      ------------  ----------  ---------------  -------------  ---
  kube-system                 local-path-provisioner-7ff9579c6-vkqxc    0 (0%)        0 (0%)      0 (0%)           0 (0%)         39m
  kube-system                 metrics-server-7b4f8b595-zpqnm            0 (0%)        0 (0%)      0 (0%)           0 (0%)         39m
  kube-system                 coredns-66c464876b-hllht                  100m (5%)     0 (0%)      70Mi (3%)        170Mi (8%)     39m
  kube-system                 svclb-traefik-7czk5                       0 (0%)        0 (0%)      0 (0%)           0 (0%)         39m
  kube-system                 traefik-5dd496474-brj2s                   0 (0%)        0 (0%)      0 (0%)           0 (0%)         39m
Allocated resources:
  (Total limits may be over 100 percent, i.e., overcommitted.)
  Resource           Requests   Limits
  --------           --------   ------
  cpu                100m (5%)  0 (0%)
  memory             70Mi (3%)  170Mi (8%)
  ephemeral-storage  0 (0%)     0 (0%)
  hugepages-1Gi      0 (0%)     0 (0%)
  hugepages-2Mi      0 (0%)     0 (0%)
Events:
  Type     Reason                   Age                From        Message
  ----     ------                   ----               ----        -------
  Normal   Starting                 39m                kubelet     Starting kubelet.
  Warning  InvalidDiskCapacity      39m                kubelet     invalid capacity 0 on image filesystem
  Normal   NodeHasSufficientPID     39m (x2 over 39m)  kubelet     Node ip-10-0-1-231.ap-northeast-2.compute.internal status is now: NodeHasSufficientPID
  Normal   NodeHasSufficientMemory  39m (x2 over 39m)  kubelet     Node ip-10-0-1-231.ap-northeast-2.compute.internal status is now: NodeHasSufficientMemory
  Normal   NodeHasNoDiskPressure    39m (x2 over 39m)  kubelet     Node ip-10-0-1-231.ap-northeast-2.compute.internal status is now: NodeHasNoDiskPressure
  Normal   NodeAllocatableEnforced  39m                kubelet     Updated Node Allocatable limit across pods
  Normal   Starting                 39m                kube-proxy  Starting kube-proxy.
  Normal   NodeReady                39m                kubelet     Node ip-10-0-1-231.ap-northeast-2.compute.internal status is now: NodeReady
    

[ec2-user@ip-10-0-1-231 ~]$ kubectl get service
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   50m

[ec2-user@ip-10-0-1-231 ~]$ kubectl describe service kubernetes
Name:              kubernetes
Namespace:         default
Labels:            component=apiserver
                   provider=kubernetes
Annotations:       <none>
Selector:          <none>
Type:              ClusterIP
IP:                10.43.0.1
Port:              https  443/TCP
TargetPort:        6443/TCP
Endpoints:         10.0.1.231:6443
Session Affinity:  None
Events:            <none>

- 가장 자주 사용하는 명령어는 아래와 같다.

kubectl exec -it <POD_NAME>

--> exec 명령어를 이용해서 특정 컨테이너에 접속할때

kubectl logs -f <POD_NAME|TYPE/NAME>

--> 로그를 확인하거나

kubectl apply -f <FILENAME>
    
--> 사용자가 정의한 상태를 apply

kubectl delete -f <FILENAME>
    
--> 사용자가 정의한 상태를 delete

- 쿠버네티스의 배포단위인 pod을 만들어보자

프로덕션 환경에서는 yaml파일로 다 정의해놓고 pod을 만드는게 일반적이다.

In [ ]:
# whoami 라는 이미지 속성을 넣어서 만들어보자.
# 네트워크가 연결되어 있지 않은 웹서비스 형태의 컨테이너를 실행시킨다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl run whoami --image subicura/whoami:1
pod/whoami created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods
NAME     READY   STATUS    RESTARTS   AGE
whoami   1/1     Running   0          39s

[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods -o wide
NAME     READY   STATUS    RESTARTS   AGE   IP          NODE                                            NOMINATED NODE   READINESS GATES
whoami   1/1     Running   0          66s   10.42.0.8   ip-10-0-1-231.ap-northeast-2.compute.internal   <none>           <none>

[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods -o yaml
apiVersion: v1
items:
- apiVersion: v1
  kind: Pod
  metadata:
    creationTimestamp: "2020-12-11T09:11:26Z"
    labels:
      run: whoami
    managedFields:
    - apiVersion: v1
      fieldsType: FieldsV1
      fieldsV1:
        f:metadata:
          f:labels:
            .: {}
            f:run: {}
        f:spec:
          f:containers:
            k:{"name":"whoami"}:
              .: {}
              f:image: {}
              f:imagePullPolicy: {}
              f:name: {}
              f:resources: {}
              f:terminationMessagePath: {}
              f:terminationMessagePolicy: {}
          f:dnsPolicy: {}
          f:enableServiceLinks: {}
          f:restartPolicy: {}
          f:schedulerName: {}
          f:securityContext: {}
          f:terminationGracePeriodSeconds: {}
      manager: kubectl-run
      operation: Update
      time: "2020-12-11T09:11:26Z"
    - apiVersion: v1
      fieldsType: FieldsV1
      fieldsV1:
        f:status:
          f:conditions:
            k:{"type":"ContainersReady"}:
              .: {}
              f:lastProbeTime: {}
              f:lastTransitionTime: {}
              f:status: {}
              f:type: {}
            k:{"type":"Initialized"}:
              .: {}
              f:lastProbeTime: {}
              f:lastTransitionTime: {}
              f:status: {}
              f:type: {}
            k:{"type":"Ready"}:
              .: {}
              f:lastProbeTime: {}
              f:lastTransitionTime: {}
              f:status: {}
              f:type: {}
          f:containerStatuses: {}
          f:hostIP: {}
          f:phase: {}
          f:podIP: {}
          f:podIPs:
            .: {}
            k:{"ip":"xxxxxxxxxxxxxxx"}:
              .: {}
              f:ip: {}
          f:startTime: {}
      manager: k3s
      operation: Update
      time: "2020-12-11T09:11:36Z"
    name: whoami
    namespace: default
    resourceVersion: "3162"
    selfLink: /api/v1/namespaces/default/pods/whoami
    uid: xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  spec:
    containers:
    - image: subicura/whoami:1
      imagePullPolicy: IfNotPresent
      name: whoami
      resources: {}
      terminationMessagePath: /dev/termination-log
      terminationMessagePolicy: File
      volumeMounts:
      - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
        name: default-token-bklrx
        readOnly: true
    dnsPolicy: ClusterFirst
    enableServiceLinks: true
    nodeName: ip-10-0-1-231.ap-northeast-2.compute.internal
    preemptionPolicy: PreemptLowerPriority
    priority: 0
    restartPolicy: Always
    schedulerName: default-scheduler
    securityContext: {}
    serviceAccount: default
    serviceAccountName: default
    terminationGracePeriodSeconds: 30
    tolerations:
    - effect: NoExecute
      key: node.kubernetes.io/not-ready
      operator: Exists
      tolerationSeconds: 300
    - effect: NoExecute
      key: node.kubernetes.io/unreachable
      operator: Exists
      tolerationSeconds: 300
    volumes:
    - name: default-token-bklrx
      secret:
        defaultMode: 420
        secretName: default-token-bklrx
  status:
    conditions:
    - lastProbeTime: null
      lastTransitionTime: "2020-12-11T09:11:26Z"
      status: "True"
      type: Initialized
    - lastProbeTime: null
      lastTransitionTime: "2020-12-11T09:11:36Z"
      status: "True"
      type: Ready
    - lastProbeTime: null
      lastTransitionTime: "2020-12-11T09:11:36Z"
      status: "True"
      type: ContainersReady
    - lastProbeTime: null
      lastTransitionTime: "2020-12-11T09:11:26Z"
      status: "True"
      type: PodScheduled
    containerStatuses:
    - containerID: containerd://1c9288f23c00f9dd1eb867afb0e6d4febf05328939370d9dbc2b512de44598ee
      image: docker.io/subicura/whoami:1
      imageID: docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9
      lastState: {}
      name: whoami
      ready: true
      restartCount: 0
      started: true
      state:
        running:
          startedAt: "2020-12-11T09:11:36Z"
    hostIP: 10.0.1.231
    phase: Running
    podIP: 10.42.0.8
    podIPs:
    - ip: 10.42.0.8
    qosClass: BestEffort
    startTime: "2020-12-11T09:11:26Z"
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""
    
[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods -o json
{
    "apiVersion": "v1",
    "items": [
        {
            "apiVersion": "v1",
            "kind": "Pod",
            "metadata": {
                "creationTimestamp": "2020-12-11T09:11:26Z",
                "labels": {
                    "run": "whoami"
                },
                "managedFields": [
                    {
                        "apiVersion": "v1",
                        "fieldsType": "FieldsV1",
                        "fieldsV1": {
                            "f:metadata": {
                                "f:labels": {
                                    ".": {},
                                    "f:run": {}
                                }
                            },
                            "f:spec": {
                                "f:containers": {
                                    "k:{\"name\":\"whoami\"}": {
                                        ".": {},
                                        "f:image": {},
                                        "f:imagePullPolicy": {},
                                        "f:name": {},
                                        "f:resources": {},
                                        "f:terminationMessagePath": {},
                                        "f:terminationMessagePolicy": {}
                                    }
                                },
                                "f:dnsPolicy": {},
                                "f:enableServiceLinks": {},
                                "f:restartPolicy": {},
                                "f:schedulerName": {},
                                "f:securityContext": {},
                                "f:terminationGracePeriodSeconds": {}
                            }
                        },
                        "manager": "kubectl-run",
                        "operation": "Update",
                        "time": "2020-12-11T09:11:26Z"
                    },
                    {
                        "apiVersion": "v1",
                        "fieldsType": "FieldsV1",
                        "fieldsV1": {
                            "f:status": {
                                "f:conditions": {
                                    "k:{\"type\":\"ContainersReady\"}": {
                                        ".": {},
                                        "f:lastProbeTime": {},
                                        "f:lastTransitionTime": {},
                                        "f:status": {},
                                        "f:type": {}
                                    },
                                    "k:{\"type\":\"Initialized\"}": {
                                        ".": {},
                                        "f:lastProbeTime": {},
                                        "f:lastTransitionTime": {},
                                        "f:status": {},
                                        "f:type": {}
                                    },
                                    "k:{\"type\":\"Ready\"}": {
                                        ".": {},
                                        "f:lastProbeTime": {},
                                        "f:lastTransitionTime": {},
                                        "f:status": {},
                                        "f:type": {}
                                    }
                                },
                                "f:containerStatuses": {},
                                "f:hostIP": {},
                                "f:phase": {},
                                "f:podIP": {},
                                "f:podIPs": {
                                    ".": {},
                                    "k:{\"ip\":\"10.42.0.8\"}": {
                                        ".": {},
                                        "f:ip": {}
                                    }
                                },
                                "f:startTime": {}
                            }
                        },
                        "manager": "k3s",
                        "operation": "Update",
                        "time": "2020-12-11T09:11:36Z"
                    }
                ],
                "name": "whoami",
                "namespace": "default",
                "resourceVersion": "3162",
                "selfLink": "/api/v1/namespaces/default/pods/whoami",
                "uid": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
            },
            "spec": {
                "containers": [
                    {
                        "image": "subicura/whoami:1",
                        "imagePullPolicy": "IfNotPresent",
                        "name": "whoami",
                        "resources": {},
                        "terminationMessagePath": "/dev/termination-log",
                        "terminationMessagePolicy": "File",
                        "volumeMounts": [
                            {
                                "mountPath": "/var/run/secrets/kubernetes.io/serviceaccount",
                                "name": "default-token-bklrx",
                                "readOnly": true
                            }
                        ]
                    }
                ],
                "dnsPolicy": "ClusterFirst",
                "enableServiceLinks": true,
                "nodeName": "ip-10-0-1-231.ap-northeast-2.compute.internal",
                "preemptionPolicy": "PreemptLowerPriority",
                "priority": 0,
                "restartPolicy": "Always",
                "schedulerName": "default-scheduler",
                "securityContext": {},
                "serviceAccount": "default",
                "serviceAccountName": "default",
                "terminationGracePeriodSeconds": 30,
                "tolerations": [
                    {
                        "effect": "NoExecute",
                        "key": "node.kubernetes.io/not-ready",
                        "operator": "Exists",
                        "tolerationSeconds": 300
                    },
                    {
                        "effect": "NoExecute",
                        "key": "node.kubernetes.io/unreachable",
                        "operator": "Exists",
                        "tolerationSeconds": 300
                    }
                ],
                "volumes": [
                    {
                        "name": "default-token-bklrx",
                        "secret": {
                            "defaultMode": 420,
                            "secretName": "default-token-bklrx"
                        }
                    }
                ]
            },
            "status": {
                "conditions": [
                    {
                        "lastProbeTime": null,
                        "lastTransitionTime": "2020-12-11T09:11:26Z",
                        "status": "True",
                        "type": "Initialized"
                    },
                    {
                        "lastProbeTime": null,
                        "lastTransitionTime": "2020-12-11T09:11:36Z",
                        "status": "True",
                        "type": "Ready"
                    },
                    {
                        "lastProbeTime": null,
                        "lastTransitionTime": "2020-12-11T09:11:36Z",
                        "status": "True",
                        "type": "ContainersReady"
                    },
                    {
                        "lastProbeTime": null,
                        "lastTransitionTime": "2020-12-11T09:11:26Z",
                        "status": "True",
                        "type": "PodScheduled"
                    }
                ],
                "containerStatuses": [
                    {
                        "containerID": "containerd://1c9288f23c00f9dd1eb867afb0e6d4febf05328939370d9dbc2b512de44598ee",
                        "image": "docker.io/subicura/whoami:1",
                        "imageID": "docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9",
                        "lastState": {},
                        "name": "whoami",
                        "ready": true,
                        "restartCount": 0,
                        "started": true,
                        "state": {
                            "running": {
                                "startedAt": "2020-12-11T09:11:36Z"
                            }
                        }
                    }
                ],
                "hostIP": "10.0.1.231",
                "phase": "Running",
                "podIP": "10.42.0.8",
                "podIPs": [
                    {
                        "ip": "10.42.0.8"
                    }
                ],
                "qosClass": "BestEffort",
                "startTime": "2020-12-11T09:11:26Z"
            }
        }
    ],
    "kind": "List",
    "metadata": {
        "resourceVersion": "",
        "selfLink": ""
    }
}

# 아래와 같은 명령어로 생성한 pod에 대해 log를 확인해보자.
[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods
NAME     READY   STATUS    RESTARTS   AGE
whoami   1/1     Running   0          6m20s

# pod에서 실행된 컨테이너의 로그를 확인할 수 있다.
# 현재서버에서 돌던 다른서버에서 돌던 알아서 api서버가 모아서 보여주기 때문에 한곳에서 모든 로그를 확인할 수 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl logs -f whoami
[2020-12-11 09:11:37] INFO  WEBrick 1.3.1
[2020-12-11 09:11:37] INFO  ruby 2.3.3 (2016-11-21) [x86_64-linux]
== Sinatra (v1.4.7) has taken the stage on 4567 for development with backup from WEBrick
[2020-12-11 09:11:37] INFO  WEBrick::HTTPServer#start: pid=7 port=4567
^C

# 직접 접속해서 들어갈 수도 있다.
# 이거 역시 이게 어떤 노드에 존재하는지 몰라도 이 명령어를 실행하면 api가 알아서 프록시 연결로 연결을 해준다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl exec -it whoami sh
kubectl exec [POD] [COMMAND] is DEPRECATED and will be removed in a future version. Use kubectl exec [POD] -- [COMMAND] instead.
/usr/src/app #
/usr/src/app # ls -al
total 16
drwxr-xr-x    1 root     root            73 Feb 10  2017 .
drwxr-xr-x    1 root     root            17 Feb 10  2017 ..
-rw-r--r--    1 root     root           193 Feb 11  2017 Dockerfile
-rw-r--r--    1 root     root            44 Feb 11  2017 Gemfile
-rw-r--r--    1 root     root           280 Feb 11  2017 Gemfile.lock
-rw-r--r--    1 root     root            73 Feb 11  2017 app.rb
/usr/src/app # cat app.rb
require 'sinatra'
require 'socket'

get '/' do
  Socket.gethostname
end

/usr/src/app #
/usr/src/app # exit

# 또한 describe 명령으로 pod의 상세정보를 확인할 수 있다.
# 여기서 중요한 내용은 Events이다.
# 이벤트 부분을 보면 스케쥴러가 assign 해줬고, 내부적으로 도커이미지가 없으니까 pulling했고,
# 컨테이너를 생성하고, 러닝까지 했다는 것을 알 수 있다.
# 쿠버네티스를 쓰다보면 뭔가 실행을 했는데 pod이 안뜨는 경우가 있다. 이따 describe pod 명령을 이용하면 좋다.
# 이벤트에 어떤 Error가 났는지 다 확인할 수 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod whoami
Name:         whoami
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 09:11:26 +0000
Labels:       run=whoami
Annotations:  <none>
Status:       Running
IP:           10.42.0.8
IPs:
  IP:  10.42.0.8
Containers:
  whoami:
    Container ID:   containerd://1c9288f23c00f9dd1eb867afb0e6d4febf05328939370d9dbc2b512de44598ee
    Image:          subicura/whoami:1
    Image ID:       docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 09:11:36 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             True
  ContainersReady   True
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type    Reason     Age   From               Message
  ----    ------     ----  ----               -------
  Normal  Scheduled  29m   default-scheduler  Successfully assigned default/whoami to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal  Pulling    29m   kubelet            Pulling image "subicura/whoami:1"
  Normal  Pulled     28m   kubelet            Successfully pulled image "subicura/whoami:1" in 9.413830827s
  Normal  Created    28m   kubelet            Created container whoami
  Normal  Started    28m   kubelet            Started container whoami


# 생성한 pod을 삭제해보자.
# 참고로 deployment라는 것은 pod을 원하는 갯수를 생성을해준다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl delete pod whoami
pod "whoami" deleted

[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods
No resources found in default namespace.

[ec2-user@ip-10-0-1-231 ~]$ kubectl get deployment
No resources found in default namespace.

- YAML 파일 예제

그러면 위에 명령어 위주로 pod을 실행했는데

yaml 형태로 정의해서 실습을 해보자.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ mkdir guide-03

[ec2-user@ip-10-0-1-231 ~]$ mkdir guide-03/task-02

[ec2-user@ip-10-0-1-231 ~]$ sudo vim guide-03/task-02/whoami-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: whoami
  labels:
    type: app
spec:
  containers:
  - name: app
    image: subicura/whoami:1
            
[ec2-user@ip-10-0-1-231 ~]$ kubectl apply -f guide-03/task-02/whoami-pod.yml
pod/whoami created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get pods
NAME     READY   STATUS    RESTARTS   AGE
whoami   1/1     Running   0          46s

[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME         READY   STATUS    RESTARTS   AGE
pod/whoami   1/1     Running   0          2m54s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   110m

pod이 생성되는 과정을 보면 컨테이너를 생성하라고 지시하고, 해당 이미지를 다운로드를 받을 것이다. 그 다음에 컨테이너가 러닝이 되는데 러닝이 된다는 것은 process가 실행이 된다는 것이고, 근데 해당 process는 실행이 되자마자 바로 application을 요청을 받을수는 없을 것이다. 예를 들어서 예를 들어서 웹서비스를 띄운다고 하면 웹이 구동되는 시간이 있기 때문이다. 따라서 컨테이너가 running하는 시간과 최종적으로 application이 running되는 시간 사이에는 약간의 겝이 있다. 아직 app이 running되지 않았는데 네트워크를 연결하게 되면 error가 날 것이다. 따라서 그 웹이 구동되는 겝동안에 상태를 체크할 수 있는 값을 줄 수 있다. app이 실행되고나서도 주기적으로 체크를 하면서 컨테이너가 정상인지 아닌지를 판단을 하고, 정상이 아니면 자체적으로 재시작을 해서 정상여부를 확인을 한다. 

이를 확인하는게 하나는 app이 실행되고나서 컨테이너가 정상인지 체크하는 livenessProbe라는게 있고, app이 준비가 잘 되었는지 체크하는 readinessProbe 라는게 있다.

아래에 작성하는 두 yaml에는 네트워크가 안되도록 작성이 된 내용이다. 따라서 해당 yaml을 실행하면 네트워크가 연결이 안되어서 error가 나는 것을 확인해보고자 한다.

- livenessProbe 예제 

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ sudo vim guide-03/task-02/whoami-pod-lp.yml
apiVersion: v1
kind: Pod
metadata:
  name: whoami-lp
  labels:
    type: app
spec:
  containers:
  - name: app
    image: subicura/whoami:1
    livenessProbe:
      httpGet:
        path: /not/exist
        port: 8080
      initialDelaySeconds: 5
      timeoutSeconds: 2 # Default 1
      periodSeconds: 5 # Defaults 10
      failureThreshold: 1 # Defaults 3
        
[ec2-user@ip-10-0-1-231 ~]$ kubectl apply -f guide-03/task-02/whoami-pod-lp.yml
pod/whoami-lp created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME            READY   STATUS    RESTARTS   AGE
pod/whoami      1/1     Running   0          17m
pod/whoami-lp   1/1     Running   0          27s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   125m

# Liveness probe 상태가 지속적으로 fail난다는 것을 확인해서 계속 재시도하다가 killing을 시킨것을 알 수 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod/whoami-lp
Name:         whoami-lp
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 10:14:50 +0000
Labels:       type=app
Annotations:  <none>
Status:       Running
IP:           10.42.0.10
IPs:
  IP:  10.42.0.10
Containers:
  app:
    Container ID:   containerd://2fbcea98f85dbd9c513ab0ea994ea07cf12427ca85157ba1a4451797cfbc3ba7
    Image:          subicura/whoami:1
    Image ID:       docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 10:16:01 +0000
    Last State:     Terminated
      Reason:       Error
      Exit Code:    137
      Started:      Fri, 11 Dec 2020 10:15:26 +0000
      Finished:     Fri, 11 Dec 2020 10:16:01 +0000
    Ready:          True
    Restart Count:  2
    Liveness:       http-get http://:8080/not/exist delay=5s timeout=2s period=5s #success=1 #failure=1
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             True
  ContainersReady   True
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type     Reason     Age               From               Message
  ----     ------     ----              ----               -------
  Normal   Scheduled  77s               default-scheduler  Successfully assigned default/whoami-lp to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal   Pulled     6s (x3 over 77s)  kubelet            Container image "subicura/whoami:1" already present on machine
  Normal   Created    6s (x3 over 77s)  kubelet            Created container app
  Normal   Started    6s (x3 over 76s)  kubelet            Started container app
  Warning  Unhealthy  1s (x3 over 71s)  kubelet            Liveness probe failed: Get "http://10.42.0.10:8080/not/exist": dial tcp 10.42.0.10:8080: connect: connection refused
  Normal   Killing    1s (x3 over 71s)  kubelet            Container app failed liveness probe, will be restarted

- readinessProbe 예제

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ sudo vim guide-03/task-02/whoami-pod-rp.yml
apiVersion: v1
kind: Pod
metadata:
  name: whoami-rp
  labels:
    type: app
spec:
  containers:
  - name: app
    image: subicura/whoami:1
    readinessProbe:
      httpGet:
        path: /not/exist
        port: 8080
      initialDelaySeconds: 5
      timeoutSeconds: 2 # Default 1
      periodSeconds: 5 # Defaults 10
      failureThreshold: 1 # Defaults 3

[ec2-user@ip-10-0-1-231 ~]$ kubectl apply -f guide-03/task-02/whoami-pod-rp.yml
pod/whoami-rp created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME            READY   STATUS             RESTARTS   AGE
pod/whoami      1/1     Running            0          45m
pod/whoami-lp   0/1     CrashLoopBackOff   13         28m
pod/whoami-rp   0/1     Running            0          8s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   153m

# pod/whoami-rp   0/1 를 봤을때 아예 뜨지 못하고 있는 모습을 확인할 수 있다.
# 상태를 아래와 같이 체크를 해보니 접속을 계속 시도하고 있는데 네트워크 접속이 안되고 있는 것을 알 수 있다.
# restart를 지정된 횟수가 넘어가 버리면 killing 된다.

[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod/whoami-rp
Name:         whoami-rp
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 10:42:56 +0000
Labels:       type=app
Annotations:  <none>
Status:       Running
IP:           10.42.0.11
IPs:
  IP:  10.42.0.11
Containers:
  app:
    Container ID:   containerd://bb4127257fb82eb13625166274cc4ba0eb31c21e917efe32f4b62a070ca0eb61
    Image:          subicura/whoami:1
    Image ID:       docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 10:42:57 +0000
    Ready:          False
    Restart Count:  0
    Readiness:      http-get http://:8080/not/exist delay=5s timeout=2s period=5s #success=1 #failure=1
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             False
  ContainersReady   False
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type     Reason     Age                   From               Message
  ----     ------     ----                  ----               -------
  Normal   Scheduled  2m55s                 default-scheduler  Successfully assigned default/whoami-rp to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal   Pulled     2m55s                 kubelet            Container image "subicura/whoami:1" already present on machine
  Normal   Created    2m55s                 kubelet            Created container app
  Normal   Started    2m55s                 kubelet            Started container app
  Warning  Unhealthy  61s (x22 over 2m46s)  kubelet            Readiness probe failed: Get "http://10.42.0.11:8080/not/exist": dial tcp 10.42.0.11:8080: connect: connection refused

- health check 예제

livenessProbe와 readinessProbe를 전반적으로 체크하는 예제이다.

위에 있던 예제들은 일부러 네트워크가 안되도록 yaml을 작성해준 것이고, 아래의 yaml에서 4567 포트는 통신이 가능한 포트로 아래와 같이 yaml을 작성해서 실행하면 정상적으로 실행이 되는 것을 확인할 수 있다.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ sudo vim guide-03/task-02/whoami-pod-health.yml
apiVersion: v1
kind: Pod
metadata:
  name: whoami-health
  labels:
    type: app
spec:
  containers:
  - name: app
    image: subicura/whoami:1
    livenessProbe:
      httpGet:
        path: /
        port: 4567
    readinessProbe:
      httpGet:
        path: /
        port: 4567

[ec2-user@ip-10-0-1-231 ~]$ kubectl apply -f guide-03/task-02/whoami-pod-health.yml
pod/whoami-health created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME                READY   STATUS             RESTARTS   AGE
pod/whoami          1/1     Running            0          51m
pod/whoami-rp       0/1     Running            0          6m52s
pod/whoami-lp       0/1     CrashLoopBackOff   15         34m
pod/whoami-health   0/1     Running            0          5s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   160m

[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod/whoami-health
Name:         whoami-health
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 10:49:43 +0000
Labels:       type=app
Annotations:  <none>
Status:       Running
IP:           10.42.0.12
IPs:
  IP:  10.42.0.12
Containers:
  app:
    Container ID:   containerd://3f6612c361698a62850363c6a20a51f075ff916d6b1f67639ad72a8ae170e3f8
    Image:          subicura/whoami:1
    Image ID:       docker.io/subicura/whoami@sha256:b560f146bf9cf80ee34bd495ce04dc9661292086781364d36125ef6fc9292fc9
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 10:49:44 +0000
    Ready:          True
    Restart Count:  0
    Liveness:       http-get http://:4567/ delay=0s timeout=1s period=10s #success=1 #failure=3
    Readiness:      http-get http://:4567/ delay=0s timeout=1s period=10s #success=1 #failure=3
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             True
  ContainersReady   True
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type    Reason     Age   From               Message
  ----    ------     ----  ----               -------
  Normal  Scheduled  21s   default-scheduler  Successfully assigned default/whoami-health to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal  Pulled     20s   kubelet            Container image "subicura/whoami:1" already present on machine
  Normal  Created    20s   kubelet            Created container app
  Normal  Started    20s   kubelet            Started container app

지금까지는 pod 안에 컨테이너가 하나 있는거만 했는데 pod 하나에 두개 이상을 넣을 수도 있다.

- multi-container 예제

pod 안에 정의된 컨테이너는 동일한 ip를 사용한다. docker-compose 같은 경우에는 같은 네트워크 대역을 쓸뿐이지 같은 아이피주소를 쓰지는 않았지만 이 pod 안에서 뜨는 여러개의 컨테이너는 동일한 ip를 쓰고 서로다른 포트를 이용을 해서 통신을 한다.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ sudo vim guide-03/task-02/whoami-pod-redis.yml
apiVersion: v1
kind: Pod
metadata:
  name: whoami-redis
  labels:
    type: stack
spec:
  containers:
  - name: app
    image: subicura/whoami-redis:1
    env:
    - name: REDIS_HOST
      value: "localhost"
  - name: db
    image: redis

[ec2-user@ip-10-0-1-231 ~]$ kubectl apply -f guide-03/task-02/whoami-pod-redis.yml
pod/whoami-redis created

[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME                READY   STATUS              RESTARTS   AGE
pod/whoami          1/1     Running             0          66m
pod/whoami-rp       0/1     Running             0          21m
pod/whoami-health   1/1     Running             0          14m
pod/whoami-lp       0/1     CrashLoopBackOff    19         49m
pod/whoami-redis    0/2     ContainerCreating   0          4s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   174m
[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod/whoami-redis
Name:         whoami-redis
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 11:04:20 +0000
Labels:       type=stack
Annotations:  <none>
Status:       Pending
IP:
IPs:          <none>
Containers:
  app:
    Container ID:
    Image:          subicura/whoami-redis:1
    Image ID:
    Port:           <none>
    Host Port:      <none>
    State:          Waiting
      Reason:       ContainerCreating
    Ready:          False
    Restart Count:  0
    Environment:
      REDIS_HOST:  localhost
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
  db:
    Container ID:
    Image:          redis
    Image ID:
    Port:           <none>
    Host Port:      <none>
    State:          Waiting
      Reason:       ContainerCreating
    Ready:          False
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             False
  ContainersReady   False
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type    Reason     Age   From               Message
  ----    ------     ----  ----               -------
  Normal  Scheduled  12s   default-scheduler  Successfully assigned default/whoami-redis to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal  Pulling    12s   kubelet            Pulling image "subicura/whoami-redis:1"
    
[ec2-user@ip-10-0-1-231 ~]$ kubectl get all
NAME                READY   STATUS             RESTARTS   AGE
pod/whoami          1/1     Running            0          67m
pod/whoami-rp       0/1     Running            0          22m
pod/whoami-health   1/1     Running            0          15m
pod/whoami-lp       0/1     CrashLoopBackOff   19         50m
pod/whoami-redis    2/2     Running            0          63s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   175m

# 이 pod 안에는 컨테이너가 두개 띄어져 있으니까 어떤 service에 대한 log를 보고 싶은지 몰라서 Error가 난다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl logs pod/whoami-redis
error: a container name must be specified for pod whoami-redis, choose one of: [app db]
        
[ec2-user@ip-10-0-1-231 ~]$ kubectl logs pod/whoami-redis app
[2020-12-11 11:04:36] INFO  WEBrick 1.3.1
[2020-12-11 11:04:36] INFO  ruby 2.3.8 (2018-10-18) [x86_64-linux-musl]
== Sinatra (v2.0.4) has taken the stage on 4567 for development with backup from WEBrick
[2020-12-11 11:04:36] INFO  WEBrick::HTTPServer#start: pid=1 port=4567
        
[ec2-user@ip-10-0-1-231 ~]$ kubectl logs pod/whoami-redis db
1:C 11 Dec 2020 11:04:44.031 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 11 Dec 2020 11:04:44.031 # Redis version=6.0.9, bits=64, commit=00000000, modified=0, pid=1, just started
1:C 11 Dec 2020 11:04:44.031 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
1:M 11 Dec 2020 11:04:44.033 * Running mode=standalone, port=6379.
1:M 11 Dec 2020 11:04:44.033 # WARNING: The TCP backlog setting of 511 cannot be enforced because /proc/sys/net/core/somaxconn is set to the lower value of 128.
1:M 11 Dec 2020 11:04:44.033 # Server initialized
1:M 11 Dec 2020 11:04:44.033 * Ready to accept connections

# 아래 명령어와 같이 컨테이너에 직접 들어갈수도 있다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl exec -it whoami-redis -c db sh
kubectl exec [POD] [COMMAND] is DEPRECATED and will be removed in a future version. Use kubectl exec [POD] -- [COMMAND] instead.
# ls -al
total 0
drwxr-xr-x 2 redis redis  6 Nov 18 08:28 .
drwxr-xr-x 1 root  root  28 Dec 11 11:04 ..
# exit

# app이라는 컨테이너에 접속했는데 여기서 redis에 접속이 가능한지 테스트를 해보자.
# redis는 별도의 컨테이너이기 때문에 접속이 가능한지, 같은 아이피를 사용중인지 체크해본다.
[ec2-user@ip-10-0-1-231 ~]$ kubectl exec -it whoami-redis -c app sh
kubectl exec [POD] [COMMAND] is DEPRECATED and will be removed in a future version. Use kubectl exec [POD] -- [COMMAND] instead.
/usr/src/app # apk add curl busybox-extras
fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz
(1/5) Installing busybox-extras (1.28.4-r3)
Executing busybox-extras-1.28.4-r3.post-install
(2/5) Installing nghttp2-libs (1.39.2-r0)
(3/5) Installing libssh2 (1.9.0-r1)
(4/5) Installing libcurl (7.61.1-r3)
(5/5) Installing curl (7.61.1-r3)
Executing busybox-1.28.4-r1.trigger
OK: 16 MiB in 35 packages
/usr/src/app # curl localhost:4567
1
/usr/src/app # curl localhost:4567
2
/usr/src/app # curl localhost:4567
3
/usr/src/app # curl localhost:4567
4
/usr/src/app # telnet localhost 6379
dbsize
:1
keys *
*1
$5
count
GET count
$1
4
quit
+OK
Connection closed by foreign host
/usr/src/app # exit
command terminated with exit code 1

[ec2-user@ip-10-0-1-231 ~]$ kubectl describe pod/whoami-redis
Name:         whoami-redis
Namespace:    default
Priority:     0
Node:         ip-10-0-1-231.ap-northeast-2.compute.internal/10.0.1.231
Start Time:   Fri, 11 Dec 2020 11:04:20 +0000
Labels:       type=stack
Annotations:  <none>
Status:       Running
IP:           10.42.0.13
IPs:
  IP:  10.42.0.13
Containers:
  app:
    Container ID:   containerd://64b0072264f511098ed05a0232b10fa1b3144bb06e1109caafc43c71c3fdfa6a
    Image:          subicura/whoami-redis:1
    Image ID:       docker.io/subicura/whoami-redis@sha256:4ebb7fcc2adc99f72aaf33f5a7a49c4f516aa413e87177542554573cb93c9454
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 11:04:35 +0000
    Ready:          True
    Restart Count:  0
    Environment:
      REDIS_HOST:  localhost
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
  db:
    Container ID:   containerd://139ed68e3d9ece9956402d818b7e243d52d6d1fdc393316b2d3966822994e17d
    Image:          redis
    Image ID:       docker.io/library/redis@sha256:e2ae53ef2864fca0a117f9b80f287e2890dd1036271dd363c20b9dfc54a8664c
    Port:           <none>
    Host Port:      <none>
    State:          Running
      Started:      Fri, 11 Dec 2020 11:04:44 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-bklrx (ro)
Conditions:
  Type              Status
  Initialized       True
  Ready             True
  ContainersReady   True
  PodScheduled      True
Volumes:
  default-token-bklrx:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-bklrx
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  <none>
Tolerations:     node.kubernetes.io/not-ready:NoExecute op=Exists for 300s
                 node.kubernetes.io/unreachable:NoExecute op=Exists for 300s
Events:
  Type    Reason     Age   From               Message
  ----    ------     ----  ----               -------
  Normal  Scheduled  15m   default-scheduler  Successfully assigned default/whoami-redis to ip-10-0-1-231.ap-northeast-2.compute.internal
  Normal  Pulling    15m   kubelet            Pulling image "subicura/whoami-redis:1"
  Normal  Pulled     15m   kubelet            Successfully pulled image "subicura/whoami-redis:1" in 13.5701249s
  Normal  Created    15m   kubelet            Created container app
  Normal  Started    15m   kubelet            Started container app
  Normal  Pulling    15m   kubelet            Pulling image "redis"
  Normal  Pulled     15m   kubelet            Successfully pulled image "redis" in 8.099270678s
  Normal  Created    15m   kubelet            Created container db
  Normal  Started    15m   kubelet            Started container db

- 지금까지 실습했던 yaml을 싹 정리해주자.

In [ ]:
[ec2-user@ip-10-0-1-231 ~]$ kubectl delete -f ./guide-03/task-02/
pod "whoami-health" deleted
pod "whoami-lp" deleted
pod "whoami-redis" deleted
pod "whoami-rp" deleted
pod "whoami" deleted